In [121]:
import numpy  as np
import pandas as pd
import math

In [122]:
data = pd.read_csv("clean_weather.csv",index_col=0)
data = data.ffill()

In [123]:
data.head

<bound method NDFrame.head of             tmax  tmin  rain  tmax_tomorrow
1970-01-01  60.0  35.0   0.0           52.0
1970-01-02  52.0  39.0   0.0           52.0
1970-01-03  52.0  35.0   0.0           53.0
1970-01-04  53.0  36.0   0.0           52.0
1970-01-05  52.0  35.0   0.0           50.0
...          ...   ...   ...            ...
2022-11-22  62.0  35.0   0.0           67.0
2022-11-23  67.0  38.0   0.0           66.0
2022-11-24  66.0  41.0   0.0           70.0
2022-11-25  70.0  39.0   0.0           62.0
2022-11-26  62.0  41.0   0.0           64.0

[13509 rows x 4 columns]>

### Forward pass

In [124]:
np.random.seed(0)

i_weights = np.random.rand(1,5) / 5 - 0.1
h_weights = np.random.rand(5,5) /5 - 0.1
h_bias = np.random.rand(1,5) / 5 - 0.1

o_weights = np.random.rand(5,1) * 50
o_bias = np.random.rand(1,1)

In [125]:
outputs =  np.zeros(3)
hiddens = np.zeros((3,5))
prev_hidden = None
sequence = data['tmax'].tail(3).to_numpy()

for i in range(3):
    x = sequence[i].reshape(1,1)
    xi = x @ i_weights #np.dot(x,i_weights)
    
    if prev_hidden is None:
        xh = xi
    else:
        xh = xi + prev_hidden @ h_weights + h_bias
        
    xh = np.tanh(xh)
    prev_hidden = xh
    hiddens[i,] = xh
    
    xo = xh @ o_weights + o_bias
    outputs[i] = xo[0][0]
    
    

### Backward pass

In [126]:
def mse(actual,predicted):
    return np.mean((actual - predicted)**2)

def mse_grad(actual,predicted):
    return (predicted - actual)




In [127]:
actuals = np.array([70,62,75])
loss_grad = mse_grad(actuals,outputs)


In [128]:
next_hidden = None

o_weight_grad, o_bias_grad, hidden_weights_grad,hidden_bias_grad, i_weight_grad = [0]*5

for i in range(2,-1,-1):
    l_grad = loss_grad[i].reshape(1,1)
    o_weight_grad += hiddens[i][:,np.newaxis] @ l_grad
    o_bias_grad += np.mean(l_grad)
    
    o_grad = l_grad @  o_weights.T
    
    if next_hidden is None:
        h_grad = o_grad
    else:
        h_grad = o_grad + next_hidden @ h_weights.T
    
    dtanh = 1 - hiddens[i,:][np.newaxis,:]**2
    h_grad = np.multiply(h_grad,dtanh)
    
    next_hidden = h_grad
    
    if i>0 :
        hidden_weights_grad += hiddens[i-1,:][:,np.newaxis] @ h_grad
        h_bias += np.mean(h_grad)
    
    i_weight_grad += sequence[i].reshape(1,1).T @ h_grad
        
        
    
    

### Updating the weights

In [129]:
lr = 1e-6
i_weights -= i_weight_grad * lr
h_weights -= hidden_weights_grad * lr
h_bias -= hidden_bias_grad * lr
o_weights -= o_weight_grad * lr
o_bias -= o_bias_grad * lr

## Complete implementation

#### Splitting data (for non time-series-data)

In [130]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

predictors = ["tmax","tmin","rain"]
target = "tmax_tomorrow"

#following method can be used for normal splitting but since we have time series data we will use differen method
#Splitting the data into train and temp parts (temp will be converted to test and validation sets)
X = data[predictors]
Y = data[target]

# First split: 70% train, 30% temp
X_train, X_temp, y_train, y_temp = train_test_split(
    X, Y, 
    test_size=0.3,
    random_state=0
)

# Second split: Split temp into validation (15%) and test (15%)
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,  # 0.5 of 30% = 15%
    random_state=0
)

# Scale the features AFTER splitting to prevent data leakage
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)  # only transform, don't fit
X_test_scaled = scaler.transform(X_test)    # only transform, don't fit




#### Splitting data (for time-series)

In [131]:
# Assuming 'date' is your date column and data is sorted by date
from sklearn.preprocessing import StandardScaler

predictors = ["tmax", "tmin", "rain"]
target = "tmax_tomorrow"

# Calculate split points
total_rows = len(data)
train_end = int(0.7 * total_rows)
valid_end = int(0.85 * total_rows)

# Split the data
X_train = data[predictors].iloc[:train_end]
y_train = data[target].iloc[:train_end]

X_valid = data[predictors].iloc[train_end:valid_end]
y_valid = data[target].iloc[train_end:valid_end]

X_test = data[predictors].iloc[valid_end:]
y_test = data[target].iloc[valid_end:]

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

# Convert to numpy arrays
X_train_scaled = np.array(X_train_scaled)
X_valid_scaled = np.array(X_valid_scaled)
X_test_scaled = np.array(X_test_scaled)

#We dont scale target data check on google for more info on why
y_train = np.array(y_train).reshape(-1,1)
y_valid = np.array(y_valid).reshape(-1,1)
y_test = np.array(y_test).reshape(-1,1)

In [132]:
X_train_scaled.shape

(9456, 3)

##### layer configuration

In [133]:
layer_conf = [{"type":"input","units":3},
              {"type": "rnn","hidden":4,"output":1}
              ]

##### Initializing weights

In [134]:
def init_params(layer_conf):
    layers = []
    for i in range(1, len(layer_conf)):
        np.random.seed(0)
        k = 1/math.sqrt(layer_conf[i]["hidden"])
        i_weight = np.random.rand(layer_conf[i-1]["units"], layer_conf[i]["hidden"]) * 2 * k - k

        h_weight = np.random.rand(layer_conf[i]["hidden"], layer_conf[i]["hidden"]) * 2 * k - k
        h_bias = np.random.rand(1, layer_conf[i]["hidden"]) * 2 * k - k

        o_weight = np.random.rand(layer_conf[i]["hidden"], layer_conf[i]["output"]) * 2 * k - k
        o_bias = np.random.rand(1, layer_conf[i]["output"]) * 2 * k - k

        layers.append(
            [i_weight, h_weight, h_bias, o_weight, o_bias]
        )
    return layers

##### Forward pass

In [135]:
def forward(x, layers):
    hiddens = []
    outputs = []
    for i in range(len(layers)):
        i_weight, h_weight, h_bias, o_weight, o_bias = layers[i]
        hidden = np.zeros((x.shape[0], i_weight.shape[1]))
        output = np.zeros((x.shape[0], o_weight.shape[1]))
        for j in range(x.shape[0]):
            input_x = x[j,:][np.newaxis,:] @ i_weight
            hidden_x = input_x + hidden[max(j-1,0),:][np.newaxis,:] @ h_weight + h_bias
            # Activation.  tanh avoids outputs getting larger and larger.
            hidden_x = np.tanh(hidden_x)
            # Store hidden for use in backprop
            hidden[j,:] = hidden_x

            # Output layer
            output_x = hidden_x @ o_weight + o_bias
            output[j,:] = output_x
        hiddens.append(hidden)
        outputs.append(output)
    return hiddens, outputs[-1]

In [136]:
def mse(actual,predicted):
    return np.mean((actual - predicted)**2)

def mse_grad(actual,predicted):
    return (predicted - actual)


In [137]:
def backward(layers, x, lr, grad, hiddens):
    for i in range(len(layers)):
        i_weight, h_weight, h_bias, o_weight, o_bias = layers[i]
        hidden = hiddens[i]
        next_h_grad = None
        i_weight_grad, h_weight_grad, h_bias_grad, o_weight_grad, o_bias_grad = [0] * 5

        for j in range(x.shape[0] - 1, -1, -1):
            # Add newaxis in the first dimension
            out_grad = grad[j,:][np.newaxis, :]

            # Output updates
            # np.newaxis creates a size 1 axis, in this case transposing matrix
            o_weight_grad += hidden[j,:][:, np.newaxis] @ out_grad
            o_bias_grad += out_grad

            # Propagate gradient to hidden unit
            h_grad = out_grad @ o_weight.T

            if j < x.shape[0] - 1:
                # Then we multiply the gradient by the hidden weights to pull gradient from next hidden state to current hidden state
                hh_grad = next_h_grad @ h_weight.T
                # Add the gradients together to combine output contribution and hidden contribution
                h_grad += hh_grad

            # Pull the gradient across the current hidden nonlinearity
            # derivative of tanh is 1 - tanh(x) ** 2
            # So we take the output of tanh (next hidden state), and plug in
            tanh_deriv = 1 - hidden[j][np.newaxis,:] ** 2

            # next_h_grad @ np.diag(tanh_deriv_next) multiplies each element of next_h_grad by the deriv
            # Effect is to pull value across nonlinearity
            h_grad = np.multiply(h_grad, tanh_deriv)

            # Store to compute h grad for previous sequence position
            next_h_grad = h_grad.copy()

            # If we're not at the very beginning
            if j > 0:
                # Multiply input from previous layer by post-nonlinearity grad at current layer
                h_weight_grad += hidden[j-1][:, np.newaxis] @ h_grad
                h_bias_grad += h_grad

            i_weight_grad += x[j,:][:,np.newaxis] @ h_grad

        # Normalize lr by number of sequence elements
        lr = lr / x.shape[0]
        i_weight -= i_weight_grad * lr
        h_weight -= h_weight_grad * lr
        h_bias -= h_bias_grad * lr
        o_weight -= o_weight_grad * lr
        o_bias -= o_bias_grad * lr
        layers[i] = [i_weight, h_weight, h_bias, o_weight, o_bias]
    return layers

##### Training loop

In [141]:
epochs = 251
lr = 1e-5

layers = init_params(layer_conf)

for epoch in range(epochs):
    sequence_len = 7
    epoch_loss = 0
    
    for j in range(X_train_scaled.shape[0]-sequence_len):
        seq_x = X_train_scaled[j:(j+sequence_len),]
        seq_y = y_train[j:(j+sequence_len),]
        hiddens, outputs = forward(seq_x,layers)
        grad = mse_grad(seq_y, outputs)
        params = backward(layers,seq_x,lr,grad,hiddens)
        epoch_loss += mse(seq_y,outputs)
        
    
    if epoch%50 == 0 :
        valid_loss = 0
        for j in range(X_valid_scaled.shape[0]-sequence_len):
            seq_x = X_train_scaled[j:(j+sequence_len)]
            seq_y = y_train[j:(j+sequence_len)]
            _, outputs = forward(seq_x,layers)
            valid_loss += mse(seq_y,outputs)
        print(f"Epoch : {epoch}, train_loss  : {epoch_loss / len(X_train_scaled)}, valid_loss : {valid_loss / len(X_valid_scaled)}")
            

Epoch : 0, train_loss  : 3127.0305785639302, valid_loss : 1843.4814193201537
Epoch : 50, train_loss  : 29.690540287695626, valid_loss : 28.07774665622211
Epoch : 100, train_loss  : 25.064541627416787, valid_loss : 26.38432076414714
Epoch : 150, train_loss  : 22.889655164162047, valid_loss : 24.997527694233533
Epoch : 200, train_loss  : 22.235640463748837, valid_loss : 24.421309853451945
Epoch : 250, train_loss  : 22.08047868301861, valid_loss : 24.34071944571677
